# BFS with `grblas`

- Problem from: http://gap.cs.berkeley.edu/benchmark.html
- Based on: https://github.com/GraphBLAS/LAGraph/blob/666b4a1fd0c7c27395dc912ecfca2cfe108dd3e4/Experimental2/Algorithm/LAGraph_BreadthFirstSearch.c
- Small test data from: https://github.com/GraphBLAS/LAGraph/tree/666b4a1fd0c7c27395dc912ecfca2cfe108dd3e4/Matrices

In [ ]:
import grblas as gb
import numpy as np
from grblas import Vector, Matrix, replace

In [ ]:
m = np.array([[0, 1, 0, 1, 1, 0, 0, 0],
              [0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 1, 0, 1, 0],
              [0, 1, 0, 0, 1, 0, 0, 0],
              [1, 1, 0, 0, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 0, 0, 0],
              [0, 0, 1, 0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0, 0, 0, 0]], dtype=bool)
# make symmetric (for now)
m = m + m.T
A = Matrix.ss.import_bitmapr(bitmap=m, values=m)
AT = A

In [ ]:
def set_sparsity(v, *flags):
    flag = 0
    for cur_flag in flags:
        flag |= cur_flag
    cflag = gb.ffi.cast('int', flag)
    res = gb.lib.GxB_Vector_Option_set(v._carg, gb.lib.GxB_SPARSITY_CONTROL, cflag)
    gb.exceptions.check_status(res, v)

In [ ]:
def get_int_type(n, atleast=None):
    if atleast:
        atleast = gb.dtypes.lookup_dtype(atleast)
    else:
        atleast = gb.dtypes.INT8
    if atleast == gb.dtypes.INT64 or n > np.iinfo(np.int32).max:
        return gb.dtypes.INT64
    elif atleast == gb.dtypes.INT32 or n > np.iinfo(np.int16).max:
        return gb.dtypes.INT32
    elif atleast == gb.dtypes.INT16 or n > np.iinfo(np.int8).max:
        return gb.dtypes.INT16
    else:
        return gb.dtypes.INT8

In [ ]:
def get_row_degrees(A):
    ones = A.apply(gb.unary.one[bool]).new()
    int_type = get_int_type(ones.nvals)
    return ones.reduce_rows(gb.monoid.plus[int_type]).new(name='degrees')

In [ ]:
def BFS_parent_push(src, A, AT):
    n = A.nrows
    int_type = get_int_type(n, atleast='INT32')  # any_secondi needs at least INT32
    semiring = gb.semiring.any_secondi[int_type]
    # frontier
    q = Vector.new(int_type, size=n, name='frontier')
    set_sparsity(q, gb.lib.GxB_SPARSE)
    q[src] = src
    # parent
    pi = Vector.new(int_type, size=n, name='parent')
    set_sparsity(pi, gb.lib.GxB_BITMAP, gb.lib.GxB_FULL)
    pi[src] = src

    for k in range(1, n):
        q(~pi.S, replace) << semiring(q @ A)
        if q.nvals == 0:
            break
        pi(q.S)[:] = q
    return pi

In [ ]:
BFS_parent_push(0, A, AT)

In [ ]:
def BFS_level_push(src, A, AT):
    n = A.nrows
    int_type = get_int_type(n)
    semiring = gb.semiring.any_pair[bool]
    # frontier
    q = gb.Vector.new(bool, size=n, name='frontier')
    set_sparsity(q, gb.lib.GxB_SPARSE)
    q[src] = True
    # level
    v = Vector.new(int_type, size=n, name='level')
    set_sparsity(v, gb.lib.GxB_BITMAP, gb.lib.GxB_FULL)
    v[src] = 0

    for k in range(1, n):
        q(~v.S, replace) << semiring(q @ A)    
        if q.nvals == 0:
            break
        v(q.S)[:] = k
    return v

In [ ]:
BFS_level_push(0, A, AT)

In [ ]:
def BFS_parent_level_push(src, A, AT):
    n = A.nrows
    int_type = get_int_type(n, atleast='INT32')  # any_secondi needs at least INT32
    semiring = gb.semiring.any_secondi[int_type]
    # frontier
    q = Vector.new(int_type, size=n, name='frontier')
    set_sparsity(q, gb.lib.GxB_SPARSE)
    q[src] = src
    # parent
    pi = Vector.new(int_type, size=n, name='parent')
    set_sparsity(pi, gb.lib.GxB_BITMAP, gb.lib.GxB_FULL)
    pi[src] = src
    # level
    v = Vector.new(int_type, size=n, name='level')
    set_sparsity(v, gb.lib.GxB_BITMAP, gb.lib.GxB_FULL)
    v[src] = 0

    for k in range(1, n):
        set_sparsity(q, gb.lib.GxB_SPARSE)
        q(~pi.S, replace) << semiring(q @ A)    
        if q.nvals == 0:
            break
        pi(q.S)[:] = q
        v(q.S)[:] = k
    return pi, v

In [ ]:
BFS_parent_level_push(0, A, AT)

In [ ]:
def BFS_parent_pushpull(src, A, AT, *, degrees=None):
    n = A.nrows
    int_type = get_int_type(n, atleast='INT32')  # any_secondi needs at least INT32
    if degrees is None:
        degrees = get_row_degrees(A)
    semiring = gb.semiring.any_secondi[int_type]
    # frontier
    q = Vector.new(int_type, size=n, name='frontier')
    q[src] = src
    # parent
    pi = Vector.new(int_type, size=n, name='parent')
    set_sparsity(pi, gb.lib.GxB_BITMAP, gb.lib.GxB_FULL)
    pi[src] = src
    # workspace for push-pull decision
    w = Vector.new(degrees.dtype, size=n, name='workspace')

    # constants for push-pull decision
    alpha = 8.0
    gamma1 = n // 8  # n_over_beta1
    gamma2 = n // 512  # n_over_beta2

    nq = 1  # number of nodes in the current level
    last_nq = 0
    edges_unexplored = A.nvals
    push_pull = True
    do_push = True
    any_pull = False
    nvisited = 1
    k = 1
    while nvisited < n:
        if push_pull:
            if do_push:
                switch_to_pull = False
                if edges_unexplored < n:
                    push_pull = False
                elif any_pull:
                    switch_to_pull = nq > last_nq and nq > gamma1
                else:
                    w(q.S, replace)[:] = degrees
                    edges_in_frontier = w.reduce().value
                    edges_unexplored -= edges_in_frontier
                    switch_to_pull = nq > last_nq and edges_in_frontier > edges_unexplored / alpha
                if switch_to_pull:
                    do_push = False
                    any_pull = True
            elif nq < last_nq and nq <= gamma2:
                do_push = True

        if do_push:
            set_sparsity(q, gb.lib.GxB_SPARSE)
            q(~pi.S, replace) << semiring(q @ A)    
        else:
            set_sparsity(q, gb.lib.GxB_BITMAP)
            q(~pi.S, replace) << semiring(AT @ q)

        last_nq = nq
        nq = q.nvals
        if nq == 0:
            break
        pi(q.S)[:] = q
        nvisited += nq
        k += 1
    return pi

In [ ]:
BFS_parent_pushpull(0, A, AT)

In [ ]:
def BFS_flexible(src, A, AT, *, degrees=None, push_pull=True, compute_parent=True, compute_level=True):
    if not compute_parent and not compute_level:
        return
    n = A.nrows
    int_type = get_int_type(n, atleast='INT32')  # any_secondi needs at least INT32
    if degrees is None:
        degrees = get_row_degrees(A)
    if compute_parent:
        semiring = gb.semiring.any_secondi[int_type]
        # frontier
        q = Vector.new(int_type, size=n, name='frontier')
        q[src] = src
        # parent
        pi = Vector.new(int_type, size=n, name='parent')
        set_sparsity(pi, gb.lib.GxB_BITMAP, gb.lib.GxB_FULL)
        pi[src] = src
        mask = pi  # ~mask is the set of unvisited nodes
    else:
        semiring = gb.semiring.any_pair[bool]
        # frontier
        q = gb.Vector.new(bool, size=n, name='frontier')
        q[src] = True
        mask = v

    if compute_level:
        # level
        v = Vector.new(int_type, size=n, name='level')
        set_sparsity(v, gb.lib.GxB_BITMAP, gb.lib.GxB_FULL)
        v[src] = 0

    if push_pull:
        w = Vector.new(degrees.dtype, size=n, name='workspace')

    # constants for push-pull decision
    alpha = 8.0
    gamma1 = n // 8  # n_over_beta1
    gamma2 = n // 512  # n_over_beta2

    nq = 1  # number of nodes in the current level
    last_nq = 0
    edges_unexplored = A.nvals
    do_push = True
    any_pull = False

    nvisited = 1
    k = 1

    while nvisited < n:
        if push_pull:
            if do_push:
                switch_to_pull = False
                if edges_unexplored < n:
                    push_pull = False
                elif any_pull:
                    switch_to_pull = nq > last_nq and nq > gamma1
                else:
                    w(q.S, replace)[:] = degrees
                    edges_in_frontier = w.reduce(gb.monoid.plus[int]).value
                    edges_unexplored -= edges_in_frontier
                    switch_to_pull = nq > last_nq and edges_in_frontier > edges_unexplored / alpha

                if switch_to_pull:
                    do_push = False
                    any_pull = True
            elif nq < last_nq and nq <= gamma2:
                do_push = True

        if do_push:
            set_sparsity(q, gb.lib.GxB_SPARSE)
            q(~mask.S, replace) << semiring(q @ A)    
        else:
            set_sparsity(q, gb.lib.GxB_BITMAP)
            q(~mask.S, replace) << semiring(AT @ q)

        last_nq = nq
        nq = q.nvals
        if nq == 0:
            break
        if compute_parent:
            pi(q.S)[:] = q
        if compute_level:
            v(q.S)[:] = k
        nvisited += nq
        k += 1
    if compute_parent and compute_level:
        return pi, v
    elif compute_parent:
        return pi
    else:
        return v

In [ ]:
BFS_flexible(0, A, AT)

In [ ]:
from glob import glob
for filename in sorted(glob('Matrices/*.mtx')):  # From LAGraph
    if filename in {'Matrices/test2003.mtx'}:  # has duplicates
        continue
    if filename in {'Matrices/jagmesh7.mtx'}:  # test not reliable
        continue
    # if filename in {'Matrices/test1000.mtx', 'Matrices/test2500.mtx'}:  # too slow
    #     continue
    A = gb.io.mmread(filename)
    if A.nrows != A.ncols:
        continue
    AT = A.T.new()
    print(filename)
    degrees = get_row_degrees(A)
    for src in range(A.nrows):
        p1 = BFS_parent_push(src, A, AT)
        l1 = BFS_level_push(src, A, AT)
        p2, l2 = BFS_parent_level_push(src, A, AT)
        p3 = BFS_parent_pushpull(src, A, AT, degrees=degrees)
        p4, l3 = BFS_flexible(src, A, AT, degrees=degrees)
        assert p1.isequal(p2)
        assert p1.isequal(p3)
        assert p1.isequal(p4)
        assert l1.isequal(l2)
        assert l1.isequal(l3)